In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from xgboost import XGBClassifier

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data = pd.read_csv(r'Parkinsons Train Data.csv', header=0)
data.head()

,Jitter(local),"Jitter(local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, db)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),Shimmer (dda),AC,NDH,HTM,Median Pitch,Mean Pitch,Standard deviation,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,status
0,1.488,0.000090,0.900,0.794,2.699,8.334,0.779,4.517,4.609,6.802,13.551,0.905905,0.119116,11.130,166.533,164.781,10.421,142.229,187.576,160,159,0.006065,0.000416,0.000,0,0.000,1
1,0.728,0.000038,0.353,0.376,1.059,5.864,0.642,2.058,3.180,7.194,6.175,0.951285,0.066758,17.403,195.252,193.289,14.773,159.515,234.505,170,169,0.005181,0.000403,2.247,0,0.000,1
2,1.220,0.000074,0.732,0.670,2.196,8.719,0.875,4.347,5.166,7.548,13.040,0.911508,0.109983,12.212,158.689,164.768,12.981,146.445,211.442,1431,1427,0.006071,0.000474,10.656,1,0.178,1
3,2.502,0.000123,1.156,1.634,3.469,13.513,1.273,5.263,8.771,16.779,15.789,0.901302,0.121982,11.384,201.997,203.471,10.853,182.713,220.230,94,92,0.004910,0.000320,0.000,0,0.000,1
4,3.509,0.000167,1.715,1.539,5.145,9.112,1.040,3.102,4.927,12.823,9.307,0.856907,0.236831,10.653,211.887,208.440,11.499,182.821,225.162,117,114,0.004757,0.000380,18.182,1,13.318,1


In [3]:
X = data.iloc[:, :-1].values
X

array([[1.48800e+00, 9.02130e-05, 9.00000e-01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [7.28000e-01, 3.76980e-05, 3.53000e-01, ..., 2.24700e+00,
        0.00000e+00, 0.00000e+00],
       [1.22000e+00, 7.40410e-05, 7.32000e-01, ..., 1.06560e+01,
        1.00000e+00, 1.78000e-01],
       ...,
       [2.91200e+00, 2.26205e-04, 1.14100e+00, ..., 2.00000e+01,
        1.00000e+00, 4.57400e+00],
       [3.78800e+00, 2.74883e-04, 1.78600e+00, ..., 1.66670e+01,
        3.00000e+00, 1.95750e+01],
       [1.80900e+00, 1.48510e-04, 6.80000e-01, ..., 1.81820e+01,
        1.00000e+00, 3.38700e+00]])

In [4]:
y = data.iloc[:, -1].values
y

array([1, 1, 1, ..., 0, 0, 0])

In [0]:
cols_11 = ['Jitter(local, absolute)','Shimmer (apq11)','AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation',
       'Minimum pitch', 'Maximum pitch', 'Number of pulses',
       'Number of periods', 'Mean period', 'Standard deviation of period',
       'Fraction of locally unvoiced frames', 'Number of voice breaks',
       'Degree of voice breaks']
data_11 = data[cols_11]

In [6]:
cols = list(data_11.columns)
model = XGBClassifier()
#Initializing RFE model
rfe = RFE(model)
#Transforming data using RFE
X_rfe = rfe.fit_transform(data_11,y)  
#Fitting the data to model
model.fit(X_rfe,y)
print(rfe.support_)
print(rfe.ranking_)
for i in range(17):
    print(cols[i], "----->", rfe.support_[i])

[ True  True False False  True  True False  True False False False  True
  True  True False False False]
[ 1  1  5  9  1  1  8  1  7  3  4  1  1  1  2 10  6]
Jitter(local, absolute) -----> True
Shimmer (apq11) -----> True
AC -----> False
NDH -----> False
HTM -----> True
Median Pitch -----> True
Mean Pitch -----> False
Standard deviation -----> True
Minimum pitch -----> False
Maximum pitch -----> False
Number of pulses -----> False
Number of periods -----> True
Mean period -----> True
Standard deviation of period -----> True
Fraction of locally unvoiced frames -----> False
Number of voice breaks -----> False
Degree of voice breaks -----> False


In [7]:
X = data_11.iloc[:, :].values
#no of features
nof_list=np.arange(1,30 )            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    from sklearn.preprocessing import StandardScaler
    standard_X = StandardScaler()
    X = standard_X.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 12)
    model = XGBClassifier()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

Optimum number of features: 17
Score with 17 features: 0.745192


In [8]:
cols = list(data_11.columns)
model = XGBClassifier()
#Initializing RFE model
rfe = RFE(model, 17)             
#Transforming data using RFE
X_rfe = rfe.fit_transform(data_11,y)  
#Fitting the data to model
model.fit(X_rfe,y)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

Index(['Jitter(local, absolute)', 'Shimmer (apq11)', 'AC', 'NDH', 'HTM',
       'Median Pitch', 'Mean Pitch', 'Standard deviation', 'Minimum pitch',
       'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period',
       'Standard deviation of period', 'Fraction of locally unvoiced frames',
       'Number of voice breaks', 'Degree of voice breaks'],
      dtype='object')


In [9]:
rfe_data = data[selected_features_rfe]
rfe_data

,"Jitter(local, absolute)",Shimmer (apq11),AC,NDH,HTM,Median Pitch,Mean Pitch,Standard deviation,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks
0,0.000090,6.802,0.905905,0.119116,11.130,166.533,164.781,10.421,142.229,187.576,160,159,0.006065,0.000416,0.000,0,0.000
1,0.000038,7.194,0.951285,0.066758,17.403,195.252,193.289,14.773,159.515,234.505,170,169,0.005181,0.000403,2.247,0,0.000
2,0.000074,7.548,0.911508,0.109983,12.212,158.689,164.768,12.981,146.445,211.442,1431,1427,0.006071,0.000474,10.656,1,0.178
3,0.000123,16.779,0.901302,0.121982,11.384,201.997,203.471,10.853,182.713,220.230,94,92,0.004910,0.000320,0.000,0,0.000
4,0.000167,12.823,0.856907,0.236831,10.653,211.887,208.440,11.499,182.821,225.162,117,114,0.004757,0.000380,18.182,1,13.318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,0.000398,8.303,0.754472,0.430569,6.798,126.492,130.350,14.387,96.145,158.591,63,60,0.007766,0.001066,24.194,2,15.113
1032,0.000133,11.928,0.880775,0.160787,11.216,128.142,132.244,7.140,121.490,143.797,47,44,0.007561,0.000430,46.053,1,25.792
1033,0.000226,14.239,0.883146,0.161736,11.035,129.727,128.388,13.593,95.921,149.641,79,77,0.007767,0.000881,20.000,1,4.574
1034,0.000275,18.818,0.765474,0.423034,6.583,126.131,135.943,46.764,90.051,451.366,75,70,0.007257,0.001714,16.667,3,19.575


In [10]:
X = rfe_data.iloc[:, :].values
X

array([[9.02130e-05, 6.80200e+00, 9.05905e-01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [3.76980e-05, 7.19400e+00, 9.51285e-01, ..., 2.24700e+00,
        0.00000e+00, 0.00000e+00],
       [7.40410e-05, 7.54800e+00, 9.11508e-01, ..., 1.06560e+01,
        1.00000e+00, 1.78000e-01],
       ...,
       [2.26205e-04, 1.42390e+01, 8.83146e-01, ..., 2.00000e+01,
        1.00000e+00, 4.57400e+00],
       [2.74883e-04, 1.88180e+01, 7.65474e-01, ..., 1.66670e+01,
        3.00000e+00, 1.95750e+01],
       [1.48510e-04, 1.10730e+01, 9.25554e-01, ..., 1.81820e+01,
        1.00000e+00, 3.38700e+00]])

In [0]:
from sklearn.preprocessing import StandardScaler
standard_X = StandardScaler()
X = standard_X.fit_transform(X)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 12)

In [13]:
model = XGBClassifier(learning_rate =0.05,
         n_estimators=250,
         max_depth=5,
         gamma=1,
         subsample=0.69,
         colsample_bytree=0.85)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.85, gamma=1,
              learning_rate=0.05, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.69, verbosity=1)

In [14]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
y_pred = model.predict(X_test)

print("Confusion Matrix")
print(confusion_matrix(y_test, y_pred))
print("\n")

print("Classfications Results")
print(classification_report(y_test, y_pred))

print("Accuracy Score")
print(accuracy_score(y_test, y_pred))

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = model, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Confusion Matrix
[[74 32]
 [23 79]]


Classfications Results
              precision    recall  f1-score   support

           0       0.76      0.70      0.73       106
           1       0.71      0.77      0.74       102

    accuracy                           0.74       208
   macro avg       0.74      0.74      0.74       208
weighted avg       0.74      0.74      0.74       208

Accuracy Score
0.7355769230769231
Accuracy: 68.72 %
Standard Deviation: 4.37 %


In [17]:
from sklearn.externals import joblib
joblib.dump(model, 'XGBoostmodel.pkl')
print("Model dumped!")

Model dumped!


In [20]:
model_columns = list(rfe_data.columns)
model_columns

['Jitter(local, absolute)',
 'Shimmer (apq11)',
 'AC',
 'NDH',
 'HTM',
 'Median Pitch',
 'Mean Pitch',
 'Standard deviation',
 'Minimum pitch',
 'Maximum pitch',
 'Number of pulses',
 'Number of periods',
 'Mean period',
 'Standard deviation of period',
 'Fraction of locally unvoiced frames',
 'Number of voice breaks',
 'Degree of voice breaks']

In [21]:
joblib.dump(model_columns, 'model_columns.pkl')
print("Models columns dumped!")

Models columns dumped!
